In [ ]:
import json, sys, copy, os
import numpy as np

matopt_repo_path = "/home/wxie/workspace/eFlesh/microstructure/matopt"
inflator_repo_path = "/home/wxie/workspace/eFlesh/microstructure/microstructure_inflators"
stack_cells_cli = os.path.join(inflator_repo_path, "build/isosurface_inflator/stack_cells")

dimensions = [1, 1, 1]
nu = 0.09
# E = 0.02
cell_size = 8
out_path = f"{dimensions[0]}x{dimensions[1]}x{dimensions[2]}_{cell_size}mm.obj"

# use return E for uniform Young's modulus
# use i,j,k to vary the Young's modulus through the dimensions
def young(i, j, k):
    # return E
    if k == 0:
        return 0.02
    elif k == 1:
        return 0.01
    elif k == 2:
        return 0.1
    # elif k == 3:
    #     return 0.0010
    # elif k == 4:
    #     return 0.0010
    # elif k == 5:
    #     return 0.0010
    # elif k == 6:
    #     return 0.0010
    # elif k == 7:
    #     return 0.0010

sys.path.insert(0, os.path.join(matopt_repo_path, 'tools/material2geometry'))

from material2geometry import Material2Geometry
mat2geo = Material2Geometry(in_path=os.path.join(matopt_repo_path, "tools/material2geometry/0646_geo_1_coeffs.txt"))

In [ ]:
patterns = []
entry = {"params": [],
"symmetry": "Cubic",
"pattern": os.path.join(inflator_repo_path, "data/patterns/3D/reference_wires/pattern0646.wire"),
"index": [0,0,0]}

for i in range(dimensions[0]):
    for j in range(dimensions[1]):
        for k in range(dimensions[2]):
            geo_params = mat2geo.evaluate(nu, young(i, j, k))
            entry["params"] = geo_params
            entry["index"] = [i,j,k]
            patterns.append(copy.deepcopy(entry))

with open("data.json", 'w') as f:
    json.dump(patterns, f)

os.system(stack_cells_cli + " -p data.json --gridSize " + str(cell_size) + " -o " + out_path + " -r 100")